# Network analysis

The NetworkInspector provides tools for quickly analyzing the created network.

First, let us create an example network:

In [1]:
import pop2net as p2n

model = p2n.Model()
creator = p2n.Creator(model=model)

for _ in range(20):
    agent = p2n.Agent(model=model)
    agent.status = "pupil"

for _ in range(4):
    agent = p2n.Agent(model=model)
    agent.status = "teacher"


class School(p2n.LocationDesigner):
    n_locations = 2
    overcrowding = False

    def weight(self, agent):
        return 0.1

    def stick_together(self, agent):
        return agent.Classroom


class PupilsInClassroom(p2n.MeltLocationDesigner):
    n_agents = 5

    def filter(self, agent):
        return agent.status == "pupil"


class TeacherInClassroom(p2n.MeltLocationDesigner):
    n_agents = 1

    def filter(self, agent):
        return agent.status == "teacher"


class Classroom(p2n.LocationDesigner):
    def melt(self):
        return PupilsInClassroom, TeacherInClassroom


creator.create_locations(
    location_designers=[
        Classroom,
        School,
    ]
)

LocationList (6 objects)

In the following, we initialize an instance of the NetworkInspector and bind it to our model:

In [2]:
inspector = p2n.NetworkInspector(model=model)

## Network visualization

The method `plot_bipartite_network()` plots the biparte network.
Agents are displayed as circles and locations are displayed as squares.
Using the arguments `agent_color` and `location_color`, agent and location attributes can be displayed as colors in the plot.
With the arguments `agent_attrs` and `location_attrs` you can add attributes that should be displayed for each node when you move the mouse pointer over it.

In the following example I display the agent attribute `status` as color and add `id` to the list of shown attribute values per agent node.

In [3]:
inspector.plot_bipartite_network(agent_color="status", agent_attrs=["id"])

Using `plot_agent_network()`, the agent-level network is shown:

In [4]:
inspector.plot_agent_network()

Using `plot_networks()`, both network types are plotted:

In [5]:
inspector.plot_networks()

## Evaluate affiliations

Using `eval_affiliations()`, the number of locations per label, the number of agents per location and the number of locations per agent can be displayed:

In [6]:
inspector.eval_affiliations()



______________________________________
Number of locations
______________________________________

location_label
Classroom    4
School       2
Name: count, dtype: int64


______________________________________
Number of agents per location
______________________________________

                mean  std   min   25%   50%   75%   max
location_label                                         
Classroom        6.0  0.0   6.0   6.0   6.0   6.0   6.0
School          12.0  0.0  12.0  12.0  12.0  12.0  12.0


______________________________________
Number of affiliated locations per agent
______________________________________

mean    2.0
std     0.0
min     2.0
25%     2.0
50%     2.0
75%     2.0
max     2.0
Name: n_affiliated_locations, dtype: float64


## 

## Network measures

Using `network_measures()`, common network measures are calculated for the agent-level network.
If there are unconnected components within the network, the network measures are calculated for each component.

In [7]:
inspector.network_measures()

[{'n_nodes': 12,
  'diameter': 0.2,
  'density': 1.0,
  'transitivity': 1.0,
  'avg_clustering': np.float64(0.3472381885547425),
  'avg_path_length': 0.1454545454545453},
 {'n_nodes': 12,
  'diameter': 0.2,
  'density': 1.0,
  'transitivity': 1.0,
  'avg_clustering': np.float64(0.3472381885547425),
  'avg_path_length': 0.1454545454545453}]